In [44]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from IPython.display import display, HTML

In [2]:
raw_df = pd.read_csv('export_2017.csv', encoding = "utf-8")#, nrows = 100000)

In [3]:
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 3.9+ GB


In [5]:
## should we have any missing customer data
cleaned_retail = raw_df.loc[pd.isnull(raw_df.CUS_N_KEY_CUSTOMER) == False]

In [6]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 4.1+ GB


Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let’s make that now.

In [10]:
item_lookup = cleaned_retail[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ART_C_PROD_NUMBER'] = item_lookup.ART_C_PROD_NUMBER.astype(int) # Encode as strings for future lookup ease
item_lookup['ART_V_ART_DESCRIPTION'] = item_lookup.ART_V_ART_DESCRIPTION.astype(str) # Encode as strings for future lookup ease

In [11]:
item_lookup.head()

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
5,451643,451643.2 Felix Häppchen in Gelee oder Leckerbi...


This can tell us what each item is, such as that ART_C_PROD_NUMBER 3481 is a  Dog Design Decke Hundeschar . Now that this has been created, we need to:

1. Group purchase quantities together by stock code and item ID
2. Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
3. Only include customers with a positive purchase total to eliminate possible errors
4. *Set up our sparse ratings matrix*

This last step is especially important if you don’t want to have unnecessary memory issues! If you think about it, our matrix is going to contain thousands of items and thousands of users with a user/item value required for every possible combination. That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only saving the locations and values of items that are not zero.

The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [12]:
cleaned_retail['CUS_N_KEY_CUSTOMER '] = cleaned_retail.CUS_N_KEY_CUSTOMER.astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['ART_C_PROD_NUMBER', 'SCM_D_QUANTITY', 'CUS_N_KEY_CUSTOMER']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index() # Group together
grouped_cleaned.SCM_D_QUANTITY.loc[grouped_cleaned.SCM_D_QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SCM_D_QUANTITY > 2') # Only get customers where purchase totals were positive

## If we look at our final resulting matrix of grouped purchases, we see the following:

grouped_purchased.head()

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,22,52735,5.0
7,22,397041,4.0
8,22,399251,3.0
12,22,447409,4.0
18,22,543556,3.0


# ToDo: 
Actually here I, perhaps, do not have to filter based on the SCM_D_QUANTITY, but I have to filter the items that have less than 5 interactions, at the same time we have to remove the users that ineracted with less than 5 different items. Check for the remaining items agan, and do it untill the DF is stabilized.


In [13]:
def threshold_likes(df, CUS_N_KEY_CUSTOMER_min, ART_C_PROD_NUMBER_min):
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        ART_C_PROD_NUMBER_counts = df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count()
        df = df[~df.CUS_N_KEY_CUSTOMER.isin(ART_C_PROD_NUMBER_counts[ART_C_PROD_NUMBER_counts < ART_C_PROD_NUMBER_min].index.tolist())]
        CUS_N_KEY_CUSTOMER_counts = df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count()
        df = df[~df.ART_C_PROD_NUMBER.isin(CUS_N_KEY_CUSTOMER_counts[CUS_N_KEY_CUSTOMER_counts < CUS_N_KEY_CUSTOMER_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count().min() >= ART_C_PROD_NUMBER_min)
    assert(df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count().min() >= CUS_N_KEY_CUSTOMER_min)
    
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [14]:
df_lim = threshold_likes(grouped_purchased, 5, 5)

Starting likes info
Number of users: 594238
Number of models: 6869
Sparsity: 0.051%
Ending likes info
Number of users: 137953
Number of models: 5564
Sparsity: 0.163%


In [15]:
df_lim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249663 entries, 0 to 11278530
Data columns (total 3 columns):
CUS_N_KEY_CUSTOMER    1249663 non-null int64
ART_C_PROD_NUMBER     1249663 non-null int64
SCM_D_QUANTITY        1249663 non-null float64
dtypes: float64(1), int64(2)
memory usage: 38.1 MB


In [17]:
customers = list(np.sort(df_lim.CUS_N_KEY_CUSTOMER.unique())) # Get our unique customers
products = list(df_lim.ART_C_PROD_NUMBER.unique()) # Get our unique products that were purchased
quantity = list(df_lim.SCM_D_QUANTITY) # All of our purchases

rows = df_lim.CUS_N_KEY_CUSTOMER.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = df_lim.ART_C_PROD_NUMBER.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [18]:
purchases_sparse

<137953x5564 sparse matrix of type '<class 'numpy.float64'>'
	with 1249663 stored elements in Compressed Sparse Row format>

In [19]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.83719241255042

# Creating a Training and Validation Set

Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set. That normally looks like this:

With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

For this example, because the time period is only 8 months and because of the purchasing type (products), it is most likely products won’t be purchased again in a short time period anyway. This will be a better test. You can see an example here:

pic

Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

As an additional check, we can compare our system to simply recommending the most popular items to every user (beating popularity is a bit difficult). This will be our baseline.

This method of testing isn’t necessarily the “correct” answer, because it depends on how you want to use the recommender system. However, it is a practical way of testing performance I will use for this example.

Now that we have a plan on how to separate our training and testing sets, let’s create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [20]:
import random

In [21]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [22]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

# SLOW Python implementation

### Imlementing ALS for Implicit feedback case

Now that we have our training and test sets finished, we can move on to implementing the algorithm. If you look at the paper of 

[Hu, Koren, and Volinsky](http://yifanhu.net/PUB/cf.pdf)

you can see the key equations will we need to implement into the algorithm. First, we have our ratings matrix which is sparse (represented by the product_train sparse matrix object). We need to turn this into a confidence matrix (from page 4):
$$
C_{ui}=1+\alpha r_{ui}
$$

Where $C_{ui}$ is the confidence matrix for our users $u$ and our items $i$. The $\alpha$ term represents a linear scaling of the rating preferences (in our case number of purchases) and the rui term is our original matrix of purchases. The paper suggests 40 (forty, Karl!) as a good starting point.

After taking the derivative of equation 3 in the paper, we can minimize the cost function for our users U:

$$
x_u=\left(Y^TC^uY+\lambda I\right)^{−1}Y^TC^u p(u)
$$

The authors note you can speed up this computation through some linear algebra that changes this equation to:
$$
x_u=\left(Y^TY+Y^T(C^u−I)Y+\lambda I\right)^{−1}Y^TC^u p(u)
$$

Notice that we can now precompute the YTY portion without having to iterate through each user u. We can derive a similar equation for our items:
$$
y_i=(X^TX+X^T(C^i−I)X+\lambda I)^{−1}X^TC^i p(i)
$$
These will be the two equations we will iterate back and forth between until they converge. We also have a regularization term λ that can help prevent overfitting during the training stage as well, along with our binarized preference matrix p which is just 1 where there was a purchase (or interaction) and zero where there was not.

Now that the math part is out of the way, we can turn this into code! Shoutout to Chris Johnson’s implicit-mf code that was a helpful guide for this. I have altered his to make things easier to understand.

In [41]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

### that is slow, oh, that is slow!

In [ ]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

We can investigate ratings for a particular user by taking the dot product between the user and item vectors (U and V). Let’s look at our first user.

In [ ]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

In [ ]:
array([ 0.00644811, -0.0014369 ,  0.00494281,  0.00027502,  0.01275582])

# Speeding Up ALS

This code in its raw form is just too slow. We have to do a lot of looping, and we haven’t taken advantage of the fact that our algorithm is embarrasingly parallel, since we could do each iteration of the item and user vectors independently. Fortunately, as I was still finishing this up, Ben Frederickson at Flipboard had perfect timing and came out with a version of ALS for Python utilizing Cython and parallelizing the code among threads. You can read his blog post about using it for finding similar music artists using matrix factorization here and his implicit library here. He claims it is even faster than Quora’s C++ QMF, but I haven’t tried theirs. All I can tell you is that it is over 1000 times faster than this bare bones pure Python version when I tested it. Install this library before you continue and follow the instructions. If you have conda installed, just do pip install implicit and you should be good to go.

First, import his library so we can utilize it for our matrix factorization.

In [23]:
import implicit

In [24]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████| 50.0/50 [00:16<00:00,  3.43it/s]


# Evaluating the Recommender System

In [25]:
from sklearn import metrics

In [26]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [27]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [28]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.906, 0.869)

We can see that our recommender system beat popularity. Our system had a mean AUC of 0.87, while the popular item benchmark had a lower AUC of 0.869. You can go back and tune the hyperparameters if you wish to see if you can get a higher AUC score. Ideally, you would have separate train, cross-validation, and test sets so that you aren’t overfitting while tuning the hyperparameters, but this setup is adequate to demonstrate how to check that the system is working.

# A Recommendation Example

In [29]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [30]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    return item_lookup.loc[item_lookup.ART_C_PROD_NUMBER.isin(prod_codes)]

We need to look these up by a customer’s ID. Looking at the list of customers:

In [77]:
customers_arr[10:105]

array([  610,   830,   883,   921,  1312,  1375,  1442,  1525,  1553,
        1611,  1682,  1775,  1814,  1836,  1921,  2016,  2081,  2236,
        2326,  2372,  2625,  2676,  2974,  3133,  3142,  3299,  3338,
        3445,  3586,  3605,  3644,  3779,  3797,  3826,  4012,  4340,
        4637,  4661,  4891,  4952,  5036,  5211,  5463,  5493,  5755,
        5855,  5909,  5987,  6096,  6174,  6481,  6489,  6675,  6835,
        6854,  6856,  6860,  6912,  7016,  7043,  7110,  7134,  7205,
        7246,  7313,  7342,  7352,  7373,  7432,  7653,  7930,  8029,
        8030,  8164,  8196,  8478,  8552,  8588,  8626,  8642,  8736,
        8752,  8767,  8810,  8829,  8844,  9093,  9229,  9431,  9758,
        9952, 10088, 10450, 10501, 10823], dtype=int64)

we can see that the first customer listed has an ID of 12346. Let’s examine their purchases from the training set.

In [33]:
get_items_purchased(22, product_train, customers_arr, products_arr, item_lookup)

[ 52735 397041 447409 543556 616867]


,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
326,397041,"397041.3 Felix ""So gut wie es aussieht"" Pouche..."
536,447409,447409.2 Vitakraft Cat Stick Mini - Lachs & Fo...
540,447409,447409.0 Vitakraft Cat Stick Mini - Ente & Kan...
1245,447409,447409.14 Vitakraft Cat Stick Mini - Truthahn ...
1246,447409,447409.15 Vitakraft Cat Stick Mini - Geflügel ...
1541,447409,447409.13 Vitakraft Cat Stick Mini - Lachs & F...
1620,52735,52735.12 Miamor Feine Filets 6 x 100 g - Thunf...
1652,447409,447409.10 Vitakraft Cat Stick Mini - Truthahn ...
1653,447409,447409.8 Vitakraft Cat Stick Mini - Lachs & Fo...
1655,447409,447409.11 Vitakraft Cat Stick Mini - Geflügel ...


In [34]:
from sklearn.preprocessing import MinMaxScaler

In [60]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        # print(code)
        rec_list.append([code, item_lookup.ART_V_ART_DESCRIPTION.loc[item_lookup.ART_C_PROD_NUMBER == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    # print (descriptions)
    final_frame = pd.DataFrame({'ART_C_PROD_NUMBER': codes, 'ART_V_ART_DESCRIPTION': descriptions}) # Create a dataframe 
    return final_frame[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']], codes # Switch order of columns around
    #return 0

In [62]:
r_df, _ = rec_items(208, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
r_df

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,508482,508482.1 Royal Canin Veterinary Diet Feline Re...
1,531249,531249.2 Kattovit Niere/Renal Pouch 6 x 85 g -...
2,471676,471676.0 Royal Canin Veterinary Diet Feline Re...
3,479505,479505.2 Kattovit Gastro Pouch 6 x 85 g - 6 x ...
4,206642,206642.4 Hill's Prescription Diet k/d Kidney C...
5,556340,556340.10 Sparpaket Kattovit Pouches 24 x 85 g...
6,508454,508454.2 Royal Canin Veterinary Diet Feline Se...
7,592299,592299.3 Animonda Integra Protect Adult Niere ...
8,592201,592201.4 Animonda Integra Protect Adult Niere ...
9,508456,508456.0 Royal Canin Veterinary Diet Feline Ur...


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what a kat food is. All it knows is the purchase history. It identified that people purchasing vet related cat food may also want to purchase sensitive cat food. 

Let us select some other user

In [39]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 221]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
1025086,345861938,81276553,0,23-JAN-17,1500488,74444244,221,1.0000,64620,46372,...,447409,447409.11 Vitakraft Cat Stick Mini - Geflügel ...,447409.11 Vitakraft Cat Stick Mini - Geflügel ...,Katzensnacks,Vitakraft,Vitakraft,Vitakraft,Vitakraft,snacks,3
1671583,345861932,81276553,0,23-JAN-17,1500466,74444252,221,1.0000,64616,46372,...,447409,447409.6 Vitakraft Cat Stick Mini - Ente & Kan...,447409.6 Vitakraft Cat Stick Mini - Ente & Kan...,Katzensnacks,Vitakraft,Vitakraft,Vitakraft,Vitakraft,snacks,3
1671899,345861928,81276553,0,23-JAN-17,1933976,107664472,221,1.0000,69844,49693,...,565453,565453.1 My Star is a Foodie - Gemischtes Pake...,565453.1 My Star is a Foodie - Gemischtes Pake...,Katzenfutter nass,★ My Star,My Star zarte Mousse,My Star zarte Mousse,MATINA,Canned Food Cat / Dog,3
1671900,345861929,81276553,0,23-JAN-17,1888821,55758654,221,1.0000,58963,43981,...,52735,52735.10 Miamor Feine Filets 6 x 100 g - Huhn ...,52735.10 Miamor Feine Filets 6 x 100 g - Huhn ...,Katzenfutter nass,Miamor,Miamor Feine Filets,Miamor Feine Filets,Finnern,Canned Food Cat / Dog,3
1671901,345861931,81276553,0,23-JAN-17,1742173,95004559,221,1.0000,67798,48682,...,518433,518433.2 Megapack Sheba Fresh & Fine Frischebe...,518433.2 Megapack Sheba Fresh & Fine Frischebe...,Katzenfutter nass,Sheba,Fresh & Fine,Fresh & Fine,Mars Petcare (Retail),Canned Food Cat / Dog,3
1671902,345861926,81276553,0,23-JAN-17,1548141,74101772,221,1.0000,64539,46365,...,347273,347273.33 Almo Nature Legend 6 x 70 g - Pazifi...,347273.33 Almo Nature Legend 6 x 70 g - Pazifi...,Katzenfutter nass,Almo Nature,Almo Nature Legend,Almo Nature Legend,Almo Nature,Canned Food Cat / Dog,3
1671903,345861936,81276553,0,23-JAN-17,1884798,55758618,221,1.0000,58960,43981,...,52735,52735.6 Miamor Feine Filets 6 x 100 g - Thunfi...,52735.6 Miamor Feine Filets 6 x 100 g - Thunfi...,Katzenfutter nass,Miamor,Miamor Feine Filets,Miamor Feine Filets,Finnern,Canned Food Cat / Dog,3
1671904,345861933,81276553,0,23-JAN-17,996339,46295104,221,1.0000,54356,41484,...,319306,319306.0 Katzenspielzeug Natural Catnip Ball -...,319306.0 Katzenspielzeug Natural Catnip Ball -...,Katzenspielzeug,Mäuse & Bälle,Bälle,Bälle,Ningbo Sincere Holding Group,"small cat / dog acc., container and local merc...",3
1671905,345861925,81276553,0,23-JAN-17,1114453,16110,221,1.0000,11754,10178,...,347273,347273.9 Almo Nature Legend 6 x 70 g - Kitten ...,347273.9 Almo Nature Legend 6 x 70 g - Kitten ...,Katzenfutter nass,Almo Nature,Almo Nature Legend,Almo Nature Legend,Almo Nature,Canned Food Cat / Dog,3
1671906,345861934,81276553,0,23-JAN-17,1500486,74444245,221,1.0000,64553,46372,...,447409,447409.10 Vitakraft Cat Stick Mini - Truthahn ...,447409.10 Vitakraft Cat Stick Mini - Truthahn ...,Katzensnacks,Vitakraft,Vitakraft,Vitakraft,Vitakraft,snacks,3


In [63]:
r_df , _ = rec_items(221, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
r_df

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,3187,3187.13 Cat's Best Original Katzenstreu - 40 l...
1,276087,276087.4 Miamor Feine Filets Naturelle 6 x 80 ...
2,174338,174338.9 Megapack Gourmet Perle 24 x 85 g - Er...
3,576075,576075.2 Multipack Sheba Classic Soup Frischeb...
4,514134,514134.4 Schesir Jelly Pouch 6 x 100 g - Thunf...
5,7786,7786.7 Miamor Feine Filets 6 x 100 g - Thunfis...
6,161599,161599.6 Schesir in Brühe 6 x 70 g - Thunfisch...
7,554599,554599.41 Schesir Katzenfutter Einzeldosen 1 x...
8,46393,46393.10 Animonda Carny Ocean 12 x 80 g - Weiß...
9,12758,12758.15 Schesir in Gelee 6 x 85 g - Thunfisch...


## Add visuals to recommendations

In [43]:
##
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [46]:
img_df = pd.read_csv('ps_id_to_img.csv', encoding = "utf-8")#, nrows = 100000)

In [47]:
name_to_url_lambda = lambda x: 'https://media.zooplus.com/bilder/' + x[x.find('.')-1] + '/140/'+ x

img_df['img_url'] = img_df['IMG_NAME'].apply(name_to_url_lambda)

In [64]:
recommendations_df , recommended_art_c_prod_numbers = rec_items(221, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
recommended_art_c_prod_numbers

[3187, 276087, 174338, 576075, 514134, 7786, 161599, 554599, 46393, 12758]

### User items

In [79]:
## DEFINE USER_ID
user_id = 10501

## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == user_id]['ART_C_PROD_NUMBER'].unique()

## get recommendations 
recommendations_df , recommended_art_c_prod_numbers = rec_items(user_id, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

In [80]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    print(img_url)

display(HTML(images))

https://media.zooplus.com/bilder/5/140/153230_sparpaket_lukullus_24_x_400_g_2017_copy_5.jpg
https://media.zooplus.com/bilder/9/140/155740_lukullus_naturkost_gemischte_probierpakete_2017_copy_9.jpg
https://media.zooplus.com/bilder/1/140/7571_rintigold_huhn_pyra_1.jpg
https://media.zooplus.com/bilder/6/140/66603_pla_concept_for_life_all_fit_3_kg_6.jpg
https://media.zooplus.com/bilder/7/140/lukullus_menu_gustico_gemischtes_probierpaket_6x400g_7.jpg
https://media.zooplus.com/bilder/2/140/67864_pla_gimcat_kaese_roll_2.jpg
https://media.zooplus.com/bilder/7/140/whiskas_8_2_7.jpg
https://media.zooplus.com/bilder/8/140/65027__65028_bd_12kg_basic_frei_130903_mg_hpr_8.jpg
https://media.zooplus.com/bilder/0/140/294176_0.jpg
https://media.zooplus.com/bilder/8/140/221463_sparpaket_schesir_in_gelee_24x85g_8.jpg
https://media.zooplus.com/bilder/5/140/51006_PLA_Happy_Dog_Flocken_Vollkost_Cereal_Flakes_10kg_5.jpg
https://media.zooplus.com/bilder/5/140/27936_pla_dokas_kaninchengetrocknet_5.jpg
https://m

### Recos for the user

In [81]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'Recommended'+'</font>'))

for code in recommended_art_c_prod_numbers:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    print(img_url)

display(HTML(images))

https://media.zooplus.com/bilder/4/140/439801_mac_s_cat_katzenfutter_sparpaket_12_x_400_g_4.jpg
https://media.zooplus.com/bilder/2/140/379076_1_1_gratis_2_x_cosma_snackies_2.jpg
https://media.zooplus.com/bilder/6/140/jetzt_testen_xxl_6.jpg
https://media.zooplus.com/bilder/5/140/_pla_leonardo_fisch_400g_5.jpg
https://media.zooplus.com/bilder/5/140/_felineporta21_12x400g_5.jpg
https://media.zooplus.com/bilder/9/140/70673_pla_granatapetdelicatessen_ente_gefluegel_400g_9.jpg
https://media.zooplus.com/bilder/9/140/574037_6_x_410_g_bozita_nassfutter_9.jpg
https://media.zooplus.com/bilder/5/140/298490_bozita_hig_rinderhack_5.jpg
https://media.zooplus.com/bilder/5/140/525680_rocco_sparpaket_chings_snack_5.jpg
https://media.zooplus.com/bilder/0/140/cookies_0.jpg
